In [ ]:
import csv
import socket
import struct
import time
import os

# Configuration
PMU_ID = 4  # Unique identifier for the PMU
PDC_IP = '192.168.60.100'
PDC_PORT = 9009
CSV_FILE = 'realistic_pmu_train.csv'
SEND_INTERVAL = 0.01  # seconds

STRUCT_FORMAT = '>Id12f'
PACKET_SIZE = struct.calcsize(STRUCT_FORMAT)

def read_csv_data(file_path):
    """Reads phasor data from a CSV file."""
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        data = [list(map(float, row[:-2])) for row in reader]
    return data

def send_with_udp_struct(phasor_data):
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    data_length = len(phasor_data)
    index = 0
    try:
        while True:
            timestamp = time.time()
            data_row = phasor_data[index]
            packet = struct.pack(STRUCT_FORMAT, PMU_ID, timestamp, *data_row)
            sock.sendto(packet, (PDC_IP, PDC_PORT))
            index = (index + 1) % data_length
            time.sleep(SEND_INTERVAL)
    except KeyboardInterrupt:
        print("UDP struct sending stopped.")
    finally:
        sock.close()


def send_with_tcp_struct(phasor_data):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect((PDC_IP, PDC_PORT))
    data_length = len(phasor_data)
    index = 0
    try:
        while True:
            timestamp = time.time()
            data_row = phasor_data[index]
            packet = struct.pack(STRUCT_FORMAT, PMU_ID, timestamp, *data_row)
            sock.sendall(packet)
            index = (index + 1) % data_length
            time.sleep(SEND_INTERVAL)
    except KeyboardInterrupt:
        print("TCP struct sending stopped.")
    finally:
        sock.close()

def read_protocol_and_mode():
    # Default values if config file not found
    protocol, mode = None, None
    config_path = r"\\DESKTOP-JKQLLNS\Users\igee2025\Desktop\config.ini"
    if os.path.exists(config_path):
        with open(config_path, "r") as f:
            lines = f.readlines()
            if len(lines) >= 1:
                protocol = lines[0].strip().split("=")[1].strip().replace('"', "").replace("'", "")
            if len(lines) >= 2:
                mode = lines[1].strip().split("=")[1].strip().replace('"', "").replace("'", "")
    return protocol, mode

def main():
    phasor_data = read_csv_data(CSV_FILE)
    print(f"Starting PMU simulation. Sending data to {PDC_IP}:{PDC_PORT} every {SEND_INTERVAL * 1000}ms.")

    # Try to read protocol/mode from config, else ask user
    protocol, mode = read_protocol_and_mode()
    if protocol is None:
        print("Choose protocol:\n1 - TCP\n2 - UDP")
        proto_input = input("Enter 1 or 2: ").strip()
        protocol = "TCP" if proto_input == "1" else "UDP"
    else:
        protocol = protocol.upper()

    print(f"Protocol: {protocol}, Mode: {mode}")

    if protocol == "UDP":
            send_with_udp_struct(phasor_data)
    elif protocol == "TCP":
        send_with_tcp_struct(phasor_data)
    else:
        print("Unknown protocol.")

if __name__ == '__main__':
    main()
